In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import pickle

import matplotlib.pyplot as plt
plt.style.use("ggplot")
%config InlineBackend.figure_format='retina'

import QuantTrading.ImpactFitting as IF
from scipy.optimize import minimize

def load_from_pickle(filename):
    path = '../pkl_dump/'
    with open(path + filename, 'rb') as f:
        return pickle.load(f)

# Load data
traded_volume_df = load_from_pickle('traded_volume_df.pkl')
px_df = load_from_pickle('px_df.pkl')
daily_stock_info_df = load_from_pickle('daily_stock_info_df.pkl')
monthly_scaling_factor = load_from_pickle('monthly_scaling_factor.pkl')
stocks = traded_volume_df.reset_index()["stock"].unique()

In [2]:
half_life = 3600
model_type = 'linear'
explanation_horizon_periods = 6
impact_px_df = IF.get_impact_state(traded_volume_df, monthly_scaling_factor, half_life, model_type)
req_stat_df = IF.impact_regression_statistics(impact_px_df, explanation_horizon_periods, px_df)
req_stat_df = req_stat_df.loc[req_stat_df["y"] >= 1e-4].copy()
req_stat_df["date"] = pd.to_datetime(req_stat_df["date"])


/Users/kelvinwu/Library/CloudStorage/OneDrive-ImperialCollegeLondon/Year 4 MSc Maths and Finance/Quant Trading and Price Impact/QuantTradingProject/QuantTrading/ImpactFitting.py:28: FutureWarning: Support for axis=1 in DataFrame.ewm is deprecated and will be removed in a future version. Use obj.T.ewm(...) instead
  cum_impact = pre_ewm.ewm(alpha=1-decay_factor, adjust=False, axis="columns").mean()  # Across columns


### Ridge model 1: global coef $\bar \lambda$, impact coef of the "stock index"

In [6]:
in_sample_month = 5
stock = 'A'

global_coef = IF.get_index_impact_coef(traded_volume_df, px_df, monthly_scaling_factor, 
                                       half_life, model_type, in_sample_month)

/Users/kelvinwu/Library/CloudStorage/OneDrive-ImperialCollegeLondon/Year 4 MSc Maths and Finance/Quant Trading and Price Impact/QuantTradingProject/QuantTrading/ImpactFitting.py:28: FutureWarning: Support for axis=1 in DataFrame.ewm is deprecated and will be removed in a future version. Use obj.T.ewm(...) instead
  cum_impact = pre_ewm.ewm(alpha=1-decay_factor, adjust=False, axis="columns").mean()  # Across columns


In [29]:
req_stat_df_in_sample_month = req_stat_df.loc[req_stat_df["date"].dt.month == in_sample_month].copy()\
                              .loc[req_stat_df["stock"] == stock]
x, y, x_valid, y_valid = IF.train_validation_split(req_stat_df_in_sample_month)

AttributeError: module 'QuantTrading.ImpactFitting' has no attribute 'train_validation_split'

In [169]:
def loss_function(params, x, y, global_coef, eta):
    coef, intercept = params  # Unpack the parameters
    predictions = coef * x + intercept
    residuals = y - predictions
    penalty = eta * (global_coef - coef) ** 2
    return (np.sum(residuals ** 2) + penalty)

In [18]:
def optimize_ridge(x, y, eta, initial_param, global_coef):
    result = minimize(fun=loss_function, x0=initial_param, 
                    args=(x, y, global_coef["beta_estimate"].iloc[0], eta), 
                    method='SLSQP', options={'max_iter': 10000, 'disp': True})
    return result.x

def validation_ridge(x_valid, y_alid, ridge_results):
    coef, intercept = ridge_results
    predictions = x.valid * coef + intercept
    return np.sum((y_valid - predictions) ** 2)

def eta_info(x, y, x_valid, y_valid, eta_list, initial_params):
    best_eta, best_params = None
    lowest_loss = np.inf
    eta_info = {eta: None for eta in eta_list}

    for eta in eta_list:
        ridge_results = optimize_ridge(x, y, eta, initial_params)
        valid_loss = validation_ridge(x_valid, y_valid, ridge_results)
        eta_info[eta] = (ridge_results, valid_loss)

    return eta_info

In [167]:
eta_list = [0.001, 0.01, 0.1, 1, 10, 100]

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.11419783121034209
            Iterations: 1
            Function evaluations: 10
            Gradient evaluations: 1


/var/folders/cb/0qjpmgyd53q970hhrqhbcvhr0000gn/T/ipykernel_73840/275889205.py:13: OptimizeWarning: Unknown solver options: max_iter
  result = minimize(fun=loss_function, x0=np.array([85, 0]),


array([8.50000000e+01, 7.48463611e-07])